In [58]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [59]:
from google.colab import files
uploaded = files.upload()


Saving visa_applications_global_with_canada_realistic (1).csv to visa_applications_global_with_canada_realistic (1) (2).csv


In [60]:
df = pd.read_csv('visa_applications_global_with_canada_realistic (1).csv')
df.head()

,application_id,destination_country,visa_type,nationality,application_date,decision_date,processing_time_days,processing_time_weeks,visa_status,processing_center,biometric_required,interview_required,application_mode
0,VISA00001,Australia,Tourist,Nigeria,2022-10-16,2022-11-22,37.0,5.29,Approved,Abuja,Yes,Yes,Online
1,VISA00002,Australia,Student,India,2022-03-12,2022-05-19,68.0,9.71,Approved,Chennai,Yes,No,NaN
2,VISA00003,Australia,Tourist,Brazil,2022-02-04,NaN,NaN,NaN,Pending,Sao Paulo,No,Yes,NaN
3,VISA00004,Canada,Work,Bangladesh,2022-04-01,2022-06-13,73.0,10.43,Approved,Dhaka,Yes,No,Offline
4,VISA00005,Canada,Student,Philippines,2022-02-08,2022-04-18,69.0,9.86,Approved,Manila,Yes,Yes,NaN


In [61]:
df.shape


(650, 13)

In [62]:
df.columns



Index(['application_id', 'destination_country', 'visa_type', 'nationality',
       'application_date', 'decision_date', 'processing_time_days',
       'processing_time_weeks', 'visa_status', 'processing_center',
       'biometric_required', 'interview_required', 'application_mode'],
      dtype='object')

In [63]:
df.isnull().sum()


,0
application_id,0
destination_country,0
visa_type,0
nationality,0
application_date,0
decision_date,85
processing_time_days,137
processing_time_weeks,137
visa_status,0
processing_center,0


In [64]:
date_cols = [
    "application_date",
    "decision_date"
]

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")


In [65]:
mask = df["processing_time_days"].isnull() & \
       df["application_date"].notnull() & \
       df["decision_date"].notnull()

df.loc[mask, "processing_time_days"] = (
    df.loc[mask, "decision_date"] - df.loc[mask, "application_date"]
).dt.days


In [66]:
df["processing_time_days"].isnull().sum()


np.int64(85)

In [67]:
df = df.dropna(subset=["processing_time_days"])


In [68]:
df["processing_time_days"].isnull().sum()

np.int64(0)

In [69]:
df = df.drop(columns=["processing_time_weeks"])


In [70]:
df["application_mode"] = df["application_mode"].fillna("Unknown")


In [71]:
df.isnull().sum()


,0
application_id,0
destination_country,0
visa_type,0
nationality,0
application_date,0
decision_date,0
processing_time_days,0
visa_status,0
processing_center,0
biometric_required,0


In [72]:
from google.colab import files
df_1=df.copy()
df_1.to_csv("visa_dataset_cleaned.csv", index=False)
files.download("visa_dataset_cleaned.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [73]:
categorical_cols = df.select_dtypes(include="object").columns
categorical_cols

Index(['application_id', 'destination_country', 'visa_type', 'nationality',
       'visa_status', 'processing_center', 'biometric_required',
       'interview_required', 'application_mode'],
      dtype='object')

In [74]:
df = df.drop(columns=["application_id"])
binary_cols = ['biometric_required', 'interview_required']

for col in binary_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0})


In [75]:
df['visa_status'].unique()


array(['Approved', 'Rejected'], dtype=object)

In [76]:
df['visa_status'] = df['visa_status'].map({'Approved': 1, 'Rejected': 0})

In [80]:
cols_to_encode = [
    'destination_country',
    'visa_type',
    'processing_center',
    'application_mode',
    'nationality'
]

# Filter the list to include only columns that are still present and of 'object' dtype
existing_categorical_cols = [
    col for col in cols_to_encode
    if col in df.columns and df[col].dtype == 'object'
]

if existing_categorical_cols:
    df = pd.get_dummies(
        df,
        columns=existing_categorical_cols,
        drop_first=True
    )
else:
    print("Skipping one-hot encoding: specified categorical columns not found or already processed.")

Skipping one-hot encoding: specified categorical columns not found or already processed.


In [78]:
df.head()

,application_date,decision_date,processing_time_days,visa_status,biometric_required,interview_required,destination_country_Canada,destination_country_UK,destination_country_USA,visa_type_Study - child,...,nationality_Bangladesh,nationality_Brazil,nationality_China,nationality_India,nationality_Japan,nationality_Nigeria,nationality_Pakistan,nationality_Philippines,nationality_United Kingdom,nationality_United States of America
0,2022-10-16,2022-11-22,37.0,1,1,1,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,2022-03-12,2022-05-19,68.0,1,1,0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,2022-04-01,2022-06-13,73.0,1,1,0,True,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,2022-02-08,2022-04-18,69.0,1,1,1,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
5,2023-05-03,2023-06-18,46.0,1,1,0,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False


In [79]:
df['visa_status'].isna().sum()


np.int64(0)

In [82]:
df.isnull().sum()


,0
application_date,0
decision_date,0
processing_time_days,0
visa_status,0
biometric_required,0
interview_required,0
destination_country_Canada,0
destination_country_UK,0
destination_country_USA,0
visa_type_Study - child,0
